## 📥 Quick Start: Download Sample Documents

If you don't have documents ready, use these commands to download real examples:

**Wikipedia Articles (Easy):**
```bash
# Download AI/ML Wikipedia articles
!wget -O ml_article.txt https://raw.githubusercontent.com/wikipedia-dump/wikipedia-articles/main/machine_learning.txt
!wget -O ai_article.txt https://raw.githubusercontent.com/wikipedia-dump/wikipedia-articles/main/artificial_intelligence.txt
```

**Scientific Papers:**
```bash
# Download arXiv papers as text
!pip install arxiv
!python -c "import arxiv; paper = next(arxiv.Search(query='machine learning').results()); print(paper.download_pdf())"
```

**Or use these pre-made sample files:**
```python
# Create sample documents automatically
sample_docs = """
Machine learning is a branch of artificial intelligence that focuses on building systems 
that can learn from data. These systems improve their performance over time without being 
explicitly programmed. Common applications include recommendation systems, image recognition,
natural language processing, and predictive analytics.

Deep learning is a subset of machine learning based on artificial neural networks with 
multiple layers. These networks can automatically learn hierarchical representations of data.
Popular architectures include CNNs for image processing, RNNs for sequential data, and 
Transformers for natural language understanding.

Reinforcement learning is a type of machine learning where an agent learns to make decisions
by interacting with an environment. The agent receives rewards or penalties and learns to 
maximize cumulative rewards. Applications include game playing, robotics, and autonomous systems.
""".strip()

with open('ml_basics.txt', 'w') as f:
    f.write(sample_docs)
    
print("✅ Sample document created: ml_basics.txt")
print("   You can now upload it in the next cell!")
```

---

# 🔧 RAG with ChromaDB - Self-Contained Notebook for Google Colab

This notebook implements a complete **RAG (Retrieval-Augmented Generation)** system using:
- **ChromaDB** as vector database
- **Sentence Transformers** for embeddings
- **Transformers (FLAN-T5)** for text generation

✅ **100% self-contained** - No external files required  
✅ **Optimized for Colab** - Works with free GPU

---

## 📚 What is RAG?

**RAG (Retrieval-Augmented Generation)** combines:
1. **Retrieval**: Search for relevant documents in a knowledge base
2. **Augmentation**: Use those documents as context
3. **Generation**: Generate responses based on the retrieved context

In [1]:
# 📦 Install dependencies (run only once)
# IMPORTANT: After installing, go to Runtime -> Restart runtime before continuing

!pip install -q torch torchvision --upgrade
!pip install -q transformers==4.38.0 --no-deps
!pip install -q tokenizers safetensors huggingface-hub
!pip install -q sentence-transformers chromadb

print("\n✅ Installation completed!")
print("⚠️  IMPORTANT: Go to Runtime -> Restart runtime before continuing with the next cell")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 1.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 149.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 145.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

### ⚠️ IMPORTANT INSTRUCTIONS:
1. Run the cell above ☝️
2. **Go to: Runtime → Restart runtime**
3. Continue with the next cell 👇

## 1️⃣ Import Libraries and Configure GPU

In [2]:
import torch
import chromadb
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from typing import List, Dict, Any, Optional
import warnings

warnings.filterwarnings('ignore')

# Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️ Device: {device}")
if device == "cuda":
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

🖥️ Device: cuda
🎮 GPU: Tesla T4
💾 GPU Memory: 15.8 GB


## 2️⃣ Create Sample Documents

Let's create a knowledge base about **Artificial Intelligence and Machine Learning**:

## 📁 Load Documents from Files (NEW)

You can load your own text documents instead of using the predefined examples.

In [ ]:
from google.colab import files
import io
import re

def chunk_text(text: str, chunk_size: int = 500, overlap: int = 50) -> list:
    """Split text into overlapping chunks."""
    words = text.split()
    chunks = []
    
    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i + chunk_size])
        if len(chunk.split()) >= 50:  # Only keep chunks with at least 50 words
            chunks.append(chunk)
    
    return chunks

def load_txt_documents(chunk_size: int = 500, overlap: int = 50):
    """
    Load and process text documents with automatic chunking.
    
    Args:
        chunk_size: Number of words per chunk (default: 500)
        overlap: Number of overlapping words between chunks (default: 50)
    """
    print("📤 Select one or more .txt files to load...")
    uploaded = files.upload()
    
    documents = []
    doc_id = 1
    
    for filename, content in uploaded.items():
        try:
            # Decode content
            text = content.decode('utf-8')
            
            # Clean text
            text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
            text = text.strip()
            
            print(f"\n📄 Processing: {filename}")
            print(f"   Original size: {len(text)} characters, {len(text.split())} words")
            
            # Chunk the text
            chunks = chunk_text(text, chunk_size, overlap)
            
            # Create document entries
            for i, chunk in enumerate(chunks, 1):
                documents.append({
                    'id': f'doc_{doc_id}',
                    'content': chunk,
                    'metadata': {
                        'source': filename,
                        'chunk': i,
                        'total_chunks': len(chunks),
                        'category': 'uploaded'
                    }
                })
                doc_id += 1
            
            print(f"   ✅ Created {len(chunks)} chunks")
        
        except Exception as e:
            print(f"   ❌ Error processing {filename}: {e}")
    
    print(f"\n📊 Total: {len(documents)} document chunks loaded")
    return documents

def load_from_url(url: str, chunk_size: int = 500):
    """Load text from a URL (Wikipedia, papers, etc.)."""
    import requests
    from bs4 import BeautifulSoup
    
    print(f"🌐 Downloading from: {url}")
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract text from paragraphs
    paragraphs = soup.find_all('p')
    text = ' '.join([p.get_text() for p in paragraphs])
    
    # Clean and chunk
    text = re.sub(r'\s+', ' ', text).strip()
    chunks = chunk_text(text, chunk_size)
    
    documents = []
    for i, chunk in enumerate(chunks, 1):
        documents.append({
            'id': f'web_doc_{i}',
            'content': chunk,
            'metadata': {
                'source': url,
                'chunk': i,
                'total_chunks': len(chunks),
                'category': 'web'
            }
        })
    
    print(f"✅ Loaded {len(documents)} chunks from URL")
    return documents

# Usage examples:
print("💡 OPTION 1 - Load your own files:")
print("   my_docs = load_txt_documents(chunk_size=500)")
print("\n💡 OPTION 2 - Load from Wikipedia:")
print("   !pip install requests beautifulsoup4")
print("   docs = load_from_url('https://en.wikipedia.org/wiki/Machine_learning')")
print("\n💡 OPTION 3 - Load from Hugging Face dataset:")
print("   # See example in next cell")

## 3️⃣ Load Real Documents

Choose one of these options to load real documents:

### 🔹 Option 1: Upload your own .txt files
Supports PDFs, Word docs, or any text file. Automatically chunks long documents.

### 🔹 Option 2: Download from Wikipedia
Load any Wikipedia article directly from URL.

### 🔹 Option 3: Use Hugging Face datasets
Access thousands of pre-built datasets with scientific papers, books, articles, etc.

### 🔹 Option 4: Download from these sources

**For AI/ML content:**
- arXiv papers: https://arxiv.org/list/cs.AI/recent
- Research papers: https://paperswithcode.com/

**For general content:**
- Wikipedia: https://en.wikipedia.org
- Project Gutenberg (books): https://www.gutenberg.org/
- Common Crawl: https://commoncrawl.org/

**💡 TIP**: Documents in **English** work best with FLAN-T5!

In [ ]:
# ============================================================
# CHOOSE ONE OF THESE OPTIONS TO LOAD REAL DOCUMENTS:
# ============================================================

# OPTION 1: Upload your own TXT files (EASIEST)
# ----------------------------------------------
# Just run this and select your files:
DOCUMENTS = load_txt_documents(chunk_size=500, overlap=50)

# OPTION 2: Load from Wikipedia (NO FILES NEEDED)
# ------------------------------------------------
# First install dependencies:
# !pip install -q requests beautifulsoup4

# Then load any Wikipedia article:
# DOCUMENTS = load_from_url('https://en.wikipedia.org/wiki/Artificial_intelligence')
# DOCUMENTS += load_from_url('https://en.wikipedia.org/wiki/Machine_learning')
# DOCUMENTS += load_from_url('https://en.wikipedia.org/wiki/Deep_learning')

# OPTION 3: Load from Hugging Face Dataset
# ------------------------------------------
# !pip install -q datasets

# from datasets import load_dataset
# dataset = load_dataset("wikipedia", "20220301.en", split="train[:100]")  # First 100 articles
# DOCUMENTS = []
# for i, item in enumerate(dataset):
#     chunks = chunk_text(item['text'], chunk_size=500)
#     for j, chunk in enumerate(chunks):
#         DOCUMENTS.append({
#             'id': f'wiki_{i}_{j}',
#             'content': chunk,
#             'metadata': {'source': item['title'], 'category': 'wikipedia'}
#         })
# print(f"✅ Loaded {len(DOCUMENTS)} chunks from Wikipedia dataset")

# OPTION 4: Load PDFs (requires extra library)
# ---------------------------------------------
# !pip install -q PyPDF2

# from PyPDF2 import PdfReader
# from google.colab import files
# 
# print("📤 Upload PDF files...")
# uploaded = files.upload()
# DOCUMENTS = []
# doc_id = 1
# 
# for filename, content in uploaded.items():
#     pdf = PdfReader(io.BytesIO(content))
#     text = ''
#     for page in pdf.pages:
#         text += page.extract_text()
#     
#     chunks = chunk_text(text, chunk_size=500)
#     for i, chunk in enumerate(chunks, 1):
#         DOCUMENTS.append({
#             'id': f'doc_{doc_id}',
#             'content': chunk,
#             'metadata': {'source': filename, 'chunk': i}
#         })
#         doc_id += 1
# 
# print(f"✅ Loaded {len(DOCUMENTS)} chunks from PDFs")

print(f"\n📚 Total documents loaded: {len(DOCUMENTS)}")
print(f"🔍 Sample document:")
print(f"   Source: {DOCUMENTS[0]['metadata']['source']}")
print(f"   Length: {len(DOCUMENTS[0]['content'])} characters")
print(f"   Preview: {DOCUMENTS[0]['content'][:200]}...")

In [4]:
# Sample knowledge base
# IMPORTANT: FLAN-T5 works better with English documents

DOCUMENTS = [
    {
        "id": "doc_1",
        "content": "Machine learning is a subset of artificial intelligence that enables computers to learn from data without being explicitly programmed. It uses algorithms to identify patterns in data and make predictions or decisions based on those patterns. Common applications include image recognition, spam filtering, and recommendation systems.",
        "metadata": {"topic": "machine_learning", "category": "fundamentals"}
    },
    {
        "id": "doc_2",
        "content": "Neural networks are computing systems inspired by the biological neural networks in animal brains. They consist of layers of interconnected nodes (neurons) that process information. Deep neural networks with many layers are called deep learning models and have revolutionized fields like computer vision and natural language processing.",
        "metadata": {"topic": "neural_networks", "category": "fundamentals"}
    },
    {
        "id": "doc_3",
        "content": "Supervised learning is a machine learning approach where the algorithm learns from labeled training data. The model is trained on input-output pairs and learns to map inputs to correct outputs. Examples include classification tasks like email spam detection and regression tasks like house price prediction.",
        "metadata": {"topic": "supervised_learning", "category": "fundamentals"}
    },
    {
        "id": "doc_4",
        "content": "Unsupervised learning is a type of machine learning that finds patterns in data without labeled examples. The algorithm tries to find structure in unlabeled data through techniques like clustering and dimensionality reduction. Common applications include customer segmentation and anomaly detection.",
        "metadata": {"topic": "unsupervised_learning", "category": "fundamentals"}
    },
    {
        "id": "doc_5",
        "content": "Natural Language Processing (NLP) is a branch of AI that helps computers understand, interpret, and generate human language. NLP techniques enable applications like machine translation, sentiment analysis, chatbots, and text summarization. Modern NLP relies heavily on deep learning models called transformers.",
        "metadata": {"topic": "nlp", "category": "applications"}
    },
    {
        "id": "doc_6",
        "content": "Computer vision is a field of AI that enables computers to interpret and understand visual information from the world. It involves tasks like image classification, object detection, facial recognition, and image segmentation. Convolutional neural networks (CNNs) are particularly effective for computer vision tasks.",
        "metadata": {"topic": "computer_vision", "category": "applications"}
    },
    {
        "id": "doc_7",
        "content": "Reinforcement learning is a machine learning paradigm where an agent learns to make decisions by interacting with an environment. The agent receives rewards or penalties based on its actions and learns to maximize cumulative rewards over time. Famous applications include game-playing AIs and robotics.",
        "metadata": {"topic": "reinforcement_learning", "category": "fundamentals"}
    },
    {
        "id": "doc_8",
        "content": "Transformers are a type of neural network architecture that has revolutionized NLP. They use self-attention mechanisms to process sequential data in parallel, making them more efficient than previous recurrent architectures. Popular transformer models include BERT, GPT, and T5.",
        "metadata": {"topic": "transformers", "category": "architectures"}
    },
    {
        "id": "doc_9",
        "content": "Transfer learning is a technique where a model trained on one task is reused as a starting point for a different but related task. This approach saves time and computational resources, as the model has already learned useful features from the first task. It's particularly effective when the new task has limited training data.",
        "metadata": {"topic": "transfer_learning", "category": "techniques"}
    },
    {
        "id": "doc_10",
        "content": "Retrieval-Augmented Generation (RAG) is a technique that enhances language models by retrieving relevant information from a knowledge base before generating responses. This approach combines the benefits of information retrieval with text generation, producing more accurate and contextually relevant outputs. RAG systems are widely used in question-answering applications.",
        "metadata": {"topic": "rag", "category": "techniques"}
    }
]

print(f"✅ {len(DOCUMENTS)} documents loaded")
print(f"📚 Topics: ML, Neural Networks, Supervised/Unsupervised Learning, NLP, Computer Vision, RL, Transformers, Transfer Learning, RAG")
print("\n🔍 Sample document:")
print(f"   {DOCUMENTS[0]['content'][:100]}...")

✅ 10 documents loaded
📚 Topics: ML, Neural Networks, Supervised/Unsupervised Learning, NLP, Computer Vision, RL, Transformers, Transfer Learning, RAG

🔍 Sample document:
   Machine learning is a subset of artificial intelligence that enables computers to learn from data wi...


## 3️⃣ Embeddings Class with Sentence Transformers

We create a class that generates embeddings using Sentence Transformers models:

In [5]:
class EmbeddingModel:
    """Generates embeddings using Sentence Transformers."""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Args:
            model_name: Name of the Sentence Transformers model
                       Options: all-MiniLM-L6-v2 (fast), all-mpnet-base-v2 (better quality)
        """
        print(f"🔄 Loading embedding model: {model_name}...")
        self.model = SentenceTransformer(model_name, device=device)
        self.dimension = self.model.get_sentence_embedding_dimension()
        print(f"✅ Model loaded - Dimension: {self.dimension}")
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Generates embeddings for a list of texts."""
        embeddings = self.model.encode(texts, convert_to_numpy=True, show_progress_bar=True)
        return embeddings.tolist()
    
    def embed_query(self, text: str) -> List[float]:
        """Generates embedding for a single text (query)."""
        embedding = self.model.encode(text, convert_to_numpy=True)
        return embedding.tolist()

# Create embedding model instance
embedding_model = EmbeddingModel("all-MiniLM-L6-v2")

🔄 Loading embedding model: all-MiniLM-L6-v2...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Model loaded - Dimension: 384


## 4️⃣ Create Vector Database with ChromaDB

We configure ChromaDB to store and search our documents:

In [6]:
class VectorStore:
    """Vector database using ChromaDB."""
    
    def __init__(self, collection_name: str = "rag_collection"):
        """Initialize ChromaDB in memory (ideal for Colab)."""
        print("🔄 Initializing ChromaDB...")
        self.client = chromadb.Client()  # In-memory client
        
        # Delete collection if exists (to allow re-running)
        try:
            self.client.delete_collection(collection_name)
        except:
            pass
        
        self.collection = self.client.create_collection(
            name=collection_name,
            metadata={"hnsw:space": "cosine"}  # Use cosine similarity
        )
        print(f"✅ Collection '{collection_name}' created")
    
    def add_documents(self, documents: List[Dict], embedding_model: EmbeddingModel):
        """Add documents to the collection."""
        texts = [doc["content"] for doc in documents]
        ids = [doc["id"] for doc in documents]
        metadatas = [doc["metadata"] for doc in documents]
        
        print(f"🔄 Generating embeddings for {len(texts)} documents...")
        embeddings = embedding_model.embed_documents(texts)
        
        self.collection.add(
            embeddings=embeddings,
            documents=texts,
            metadatas=metadatas,
            ids=ids
        )
        print(f"✅ {len(documents)} documents added to the database")
    
    def search(self, query: str, embedding_model: EmbeddingModel, 
               n_results: int = 3, filter_metadata: Optional[Dict] = None) -> Dict:
        """Search for documents similar to the query."""
        query_embedding = embedding_model.embed_query(query)
        
        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=n_results,
            where=filter_metadata,
            include=["documents", "metadatas", "distances"]
        )
        return results
    
    def get_stats(self) -> Dict:
        """Get collection statistics."""
        return {
            "total_documents": self.collection.count(),
            "collection_name": self.collection.name
        }

# Create vector database and add documents
vector_store = VectorStore("ai_knowledge_base")
vector_store.add_documents(DOCUMENTS, embedding_model)
print(f"\n📊 Statistics: {vector_store.get_stats()}")

🔄 Initializing ChromaDB...
✅ Collection 'ai_knowledge_base' created
🔄 Generating embeddings for 10 documents...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ 10 documents added to the database

📊 Statistics: {'total_documents': 10, 'collection_name': 'ai_knowledge_base'}


## 5️⃣ Test Semantic Search

Let's see how document retrieval works:

In [7]:
# Test semantic search
test_queries = [
    "What are transformers and how do they work?",
    "How can I improve a pre-trained model?",
    "What is RAG and what is it used for?"
]

for query in test_queries:
    print(f"\n🔍 Query: '{query}'")
    print("-" * 60)
    
    results = vector_store.search(query, embedding_model, n_results=2)
    
    for i, (doc, metadata, distance) in enumerate(zip(
        results['documents'][0], 
        results['metadatas'][0], 
        results['distances'][0]
    )):
        similarity = 1 - distance  # Convert distance to similarity
        print(f"\n  📄 Result {i+1} (similarity: {similarity:.3f})")
        print(f"     Topic: {metadata['topic']}")
        print(f"     Text: {doc[:150]}...")


🔍 Query: 'What are transformers and how do they work?'
------------------------------------------------------------

  📄 Result 1 (similarity: 0.569)
     Topic: transformers
     Text: Transformers are a type of neural network architecture that has revolutionized NLP. They use self-attention mechanisms to process sequential data in p...

  📄 Result 2 (similarity: 0.246)
     Topic: transfer_learning
     Text: Transfer learning is a technique where a model trained on one task is reused as a starting point for a different but related task. This approach saves...

🔍 Query: 'How can I improve a pre-trained model?'
------------------------------------------------------------

  📄 Result 1 (similarity: 0.312)
     Topic: transfer_learning
     Text: Transfer learning is a technique where a model trained on one task is reused as a starting point for a different but related task. This approach saves...

  📄 Result 2 (similarity: 0.224)
     Topic: transformers
     Text: Transformers are a 

## 6️⃣ Generator Model (FLAN-T5)

We load a text generation model to answer questions:

In [8]:
class TextGenerator:
    """Text generator using FLAN-T5."""
    
    def __init__(self, model_name: str = "google/flan-t5-base"):
        """
        Args:
            model_name: Hugging Face model
                       Options: google/flan-t5-small (fast), google/flan-t5-base (better)
        """
        print(f"🔄 Loading generator model: {model_name}...")
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto" if device == "cuda" else None
        )
        
        if device == "cpu":
            self.model = self.model.to(device)
        
        print(f"✅ Model loaded on {device}")
    
    def generate(self, prompt: str, max_length: int = 256) -> str:
        """Generate text from a prompt."""
        inputs = self.tokenizer(
            prompt, 
            return_tensors="pt", 
            max_length=512, 
            truncation=True
        ).to(self.model.device)
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_length=max_length,
                min_length=20,
                num_beams=4,
                early_stopping=True,
                no_repeat_ngram_size=3,
                do_sample=False  # False for more coherence
            )
        
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

# Load generator model
generator = TextGenerator("google/flan-t5-base")

print("\n💡 TIP: If responses are not good, you can:")
print("   1. Use a larger model: google/flan-t5-large")
print("   2. Add more relevant documents")
print("   3. Ask more specific questions")

🔄 Loading generator model: google/flan-t5-base...


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

✅ Model loaded on cuda

💡 TIP: If responses are not good, you can:
   1. Use a larger model: google/flan-t5-large
   2. Add more relevant documents
   3. Ask more specific questions


## 7️⃣ Complete RAG System

We combine everything into a functional RAG pipeline:

In [9]:
class RAGSystem:
    """Complete RAG system: Retrieval + Augmentation + Generation."""
    
    def __init__(self, vector_store: VectorStore, embedding_model: EmbeddingModel, 
                 generator: TextGenerator):
        self.vector_store = vector_store
        self.embedding_model = embedding_model
        self.generator = generator
    
    def build_prompt(self, question: str, context_docs: List[str]) -> str:
        """Build the prompt with context for the model."""
        # FLAN-T5 works better with simpler and more direct prompts
        context = "\n".join(context_docs)
        
        # Simplified format that works better with FLAN-T5
        prompt = f"""Answer the question based on the context below.

Context: {context}

Question: {question}
Answer:"""
        return prompt
    
    def query(self, question: str, n_docs: int = 3, verbose: bool = True) -> Dict[str, Any]:
        """
        Execute the complete RAG pipeline.
        
        Args:
            question: User's question
            n_docs: Number of documents to retrieve
            verbose: Whether to show detailed information
        
        Returns:
            Dictionary with response, context and metadata
        """
        # 1. RETRIEVAL - Search for relevant documents
        if verbose:
            print(f"🔍 Searching for relevant documents...")
        
        search_results = self.vector_store.search(
            question, self.embedding_model, n_results=n_docs
        )
        
        retrieved_docs = search_results['documents'][0]
        metadatas = search_results['metadatas'][0]
        distances = search_results['distances'][0]
        
        if verbose:
            print(f"📄 {len(retrieved_docs)} documents found")
        
        # 2. AUGMENTATION - Build prompt with context
        prompt = self.build_prompt(question, retrieved_docs)
        
        if verbose:
            print(f"📝 Prompt built ({len(prompt)} characters)")
        
        # 3. GENERATION - Generate response
        if verbose:
            print(f"🤖 Generating response...")
        
        response = self.generator.generate(prompt, max_length=128)
        
        return {
            "question": question,
            "answer": response,
            "retrieved_documents": retrieved_docs,
            "metadata": metadatas,
            "similarity_scores": [1 - d for d in distances],
            "prompt": prompt
        }

# Create RAG system
rag = RAGSystem(vector_store, embedding_model, generator)
print("✅ RAG System initialized and ready!")
print("\n⚠️ IMPORTANT: FLAN-T5 works better with:")
print("   • Questions in ENGLISH")
print("   • Documents in ENGLISH")
print("   • Direct and simple questions")

✅ RAG System initialized and ready!

⚠️ IMPORTANT: FLAN-T5 works better with:
   • Questions in ENGLISH
   • Documents in ENGLISH
   • Direct and simple questions


## 8️⃣ Test the RAG System

Let's ask some questions!

In [10]:
# Test questions (in English for better FLAN-T5 performance)
questions = [
    "What is machine learning?",
    "How do neural networks work?",
    "What is the difference between supervised and unsupervised learning?",
    "What are transformers in NLP?",
    "What is RAG and how does it work?"
]

In [11]:
# Run RAG for each question
for i, question in enumerate(questions, 1):
    print(f"\n{'='*70}")
    print(f"📌 QUESTION {i}: {question}")
    print('='*70)
    
    result = rag.query(question, n_docs=2, verbose=True)
    
    print(f"\n💡 ANSWER:")
    print(f"   {result['answer']}")
    
    print(f"\n📚 Documents used:")
    for j, (doc, score) in enumerate(zip(result['retrieved_documents'], 
                                          result['similarity_scores'])):
        print(f"   [{j+1}] (similarity: {score:.3f}) {doc[:100]}...")


📌 QUESTION 1: What is machine learning?
🔍 Searching for relevant documents...
📄 2 documents found
📝 Prompt built (743 characters)
🤖 Generating response...

💡 ANSWER:
   enables computers to learn without being explicitly programmed to learn from data without being explicit programmed

📚 Documents used:
   [1] (similarity: 0.802) Machine learning is a subset of artificial intelligence that enables computers to learn from data wi...
   [2] (similarity: 0.685) Supervised learning is a machine learning approach where the algorithm learns from labeled training ...

📌 QUESTION 2: How do neural networks work?
🔍 Searching for relevant documents...
📄 2 documents found
📝 Prompt built (759 characters)
🤖 Generating response...

💡 ANSWER:
   layers of interconnected nodes (neurons) that process information and have revolutionized fields like

📚 Documents used:
   [1] (similarity: 0.741) Neural networks are computing systems inspired by the biological neural networks in animal brains. T...
   [2] (

## 9️⃣ Interactive Mode

Ask your own questions:

In [12]:
# 🎯 Write your own question here (in English for best results)
my_question = "What is reinforcement learning?"

print(f"❓ Your question: {my_question}\n")
result = rag.query(my_question, n_docs=3, verbose=True)

print(f"\n{'='*60}")
print(f"💡 ANSWER: {result['answer']}")
print('='*60)

❓ Your question: What is reinforcement learning?

🔍 Searching for relevant documents...
📄 3 documents found
📝 Prompt built (1052 characters)
🤖 Generating response...

💡 ANSWER: an agent learns to make decisions by interacting with an environment. Reinforcement learning is a machine learning


## 🔟 Add New Documents

You can expand the knowledge base:

In [13]:
# Add new documents
new_docs = [
    {
        "id": "doc_11",
        "content": """AI agents are autonomous systems that can plan, reason, and execute 
        complex tasks. They use LLMs as their brain, combined with external tools, memory, 
        and reflection capabilities. Popular frameworks include LangChain Agents, AutoGPT, 
        and CrewAI.""",
        "metadata": {"topic": "agents", "category": "techniques"}
    },
    {
        "id": "doc_12",
        "content": """Prompting is the art of designing effective instructions for LLMs.
        Techniques include: zero-shot (no examples), few-shot (with examples), 
        chain-of-thought (step-by-step reasoning), and role prompting (assigning roles).
        A good prompt is clear, specific, and provides relevant context.""",
        "metadata": {"topic": "prompting", "category": "techniques"}
    }
]

vector_store.add_documents(new_docs, embedding_model)
print(f"\n📊 Total documents: {vector_store.get_stats()['total_documents']}")

🔄 Generating embeddings for 2 documents...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ 2 documents added to the database

📊 Total documents: 12


In [14]:
# Test with the new documents
agents_question = "What are AI agents and what frameworks exist?"
print(f"❓ {agents_question}\n")

result = rag.query(agents_question, n_docs=2, verbose=True)
print(f"\n💡 ANSWER: {result['answer']}")

❓ What are AI agents and what frameworks exist?

🔍 Searching for relevant documents...
📄 2 documents found
📝 Prompt built (698 characters)
🤖 Generating response...

💡 ANSWER: autonomous systems that can plan, reason, and execute complex tasks. They use LLMs as their brain


## 1️⃣1️⃣ Search with Metadata Filters

You can filter documents by category or topic:

In [15]:
# Search only in documents of category "techniques"
print("🔍 Filtered search by category 'techniques':\n")

filtered_results = vector_store.search(
    "How to improve AI models?",
    embedding_model,
    n_results=3,
    filter_metadata={"category": "techniques"}
)

for doc, meta, dist in zip(
    filtered_results['documents'][0],
    filtered_results['metadatas'][0],
    filtered_results['distances'][0]
):
    print(f"📄 Topic: {meta['topic']} | Similarity: {1-dist:.3f}")
    print(f"   {doc[:120]}...\n")

🔍 Filtered search by category 'techniques':

📄 Topic: agents | Similarity: 0.422
   AI agents are autonomous systems that can plan, reason, and execute 
        complex tasks. They use LLMs as their brain...

📄 Topic: transfer_learning | Similarity: 0.237
   Transfer learning is a technique where a model trained on one task is reused as a starting point for a different but rel...

📄 Topic: prompting | Similarity: 0.188
   Prompting is the art of designing effective instructions for LLMs.
        Techniques include: zero-shot (no examples), ...



## 1️⃣2️⃣ RAG System Evaluation

Basic metrics to evaluate quality:

In [16]:
def evaluate_retrieval(query: str, expected_topic: str, n_results: int = 3) -> Dict:
    """Evaluate if the retriever finds documents of the expected topic."""
    results = vector_store.search(query, embedding_model, n_results=n_results)
    
    topics_found = [m['topic'] for m in results['metadatas'][0]]
    hit = expected_topic in topics_found
    position = topics_found.index(expected_topic) + 1 if hit else -1
    
    return {
        "query": query,
        "expected": expected_topic,
        "found_topics": topics_found,
        "hit": hit,
        "position": position,
        "mrr": 1/position if hit else 0  # Mean Reciprocal Rank
    }

# Test cases
test_cases = [
    ("What are transformers?", "transformers"),
    ("How does neural network learning work?", "neural_networks"),
    ("What is transfer learning?", "transfer_learning"),
    ("What are the types of machine learning?", "machine_learning"),
    ("What is RAG?", "rag"),
]

print("📊 RETRIEVER EVALUATION\n" + "="*50)
eval_results = [evaluate_retrieval(q, t) for q, t in test_cases]

total_hits = sum(1 for r in eval_results if r['hit'])
avg_mrr = sum(r['mrr'] for r in eval_results) / len(eval_results)

for r in eval_results:
    status = "✅" if r['hit'] else "❌"
    print(f"{status} '{r['query'][:40]}...'")
    print(f"   Expected: {r['expected']} | Found: {r['found_topics']}")

print(f"\n📈 METRICS:")
print(f"   Accuracy: {total_hits}/{len(test_cases)} ({total_hits/len(test_cases)*100:.1f}%)")
print(f"   MRR: {avg_mrr:.3f}")

📊 RETRIEVER EVALUATION
✅ 'What are transformers?...'
   Expected: transformers | Found: ['transformers', 'transfer_learning', 'nlp']
✅ 'How does neural network learning work?...'
   Expected: neural_networks | Found: ['neural_networks', 'supervised_learning', 'machine_learning']
✅ 'What is transfer learning?...'
   Expected: transfer_learning | Found: ['transfer_learning', 'supervised_learning', 'reinforcement_learning']
✅ 'What are the types of machine learning?...'
   Expected: machine_learning | Found: ['machine_learning', 'supervised_learning', 'unsupervised_learning']
✅ 'What is RAG?...'
   Expected: rag | Found: ['rag', 'transfer_learning', 'transformers']

📈 METRICS:
   Accuracy: 5/5 (100.0%)
   MRR: 1.000


## 🎉 Notebook Completed!

### Summary of what we built:

| Component | Description |
|-----------|-------------|
| **EmbeddingModel** | Generates semantic vectors with Sentence Transformers |
| **VectorStore** | Vector database with ChromaDB |
| **TextGenerator** | FLAN-T5 model for generating responses |
| **RAGSystem** | Complete pipeline: Retrieval → Augmentation → Generation |

### 🚀 Next steps:
1. Add more documents to the knowledge base
2. Try larger models (flan-t5-large, flan-t5-xl)
3. Implement chunking for long documents
4. Add reranking to improve precision
5. Connect with external APIs (OpenAI, Anthropic)

---
**Created for the AI Mastery course** 🎓

## 🔧 Troubleshooting

### If responses are not good:

**Option 1: Try a larger model**
```python
generator = TextGenerator("google/flan-t5-large")  # Better quality but slower
rag = RAGSystem(vector_store, embedding_model, generator)
```

**Option 2: Adjust generation parameters**
```python
# In the TextGenerator class, modify generate():
outputs = self.model.generate(
    **inputs,
    max_length=256,        # Increase for longer responses
    min_length=30,         # Minimum words
    num_beams=6,           # More beams = better quality but slower
    temperature=0.5,       # Between 0 and 1 (0=deterministic, 1=creative)
    do_sample=True         # True for more variety
)
```

**Option 3: Improve the prompt**
```python
# Modify build_prompt() in RAGSystem:
prompt = f"""Based on the following context, answer the question concisely.

Context:
{context}

Question: {question}

Provide a clear, factual answer based only on the context above:
Answer:"""
```

**Option 4: Use retrieval only (without generation)**
```python
# If generation fails, at least you can see the relevant documents:
results = vector_store.search("your question", embedding_model, n_results=3)
for doc in results['documents'][0]:
    print(doc)
```